In [1]:
import pandas as pd
data = pd.read_csv("processed_dataset.csv")
data.head()

,Unnamed: 0,created_at,full_text,likes_count,mentioning_users/0,mentioning_users/1,mentioning_users/2,mentioning_users/3,mentioning_users/4,mentioning_users/5,...,tweet_hashtags/23,tweet_id,tweet_links/0,tweet_links/1,tweet_links/2,tweet_links/3,url,user_is_verified,user_name,language
0,1,Sun Aug 20 15:11:00 +00:00 2023,"Le dimanche c'est #D, comme #Destitution ???? ...",13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1693279617707339931,NaN,NaN,NaN,NaN,https://twitter.com/AHcCamille/status/16932796...,False,31be7426,fr
1,2,Thu Aug 10 18:12:00 +00:00 2023,#propagande!! #Vaccination #gardasil,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1689701346520248320,NaN,NaN,NaN,NaN,https://twitter.com/Maren31_Uhl/status/1689701...,False,5c3a8cbb,it
2,3,Sat May 20 19:20:00 +00:00 2023,Learn about Gardasil HPV Vaccine Multidistrict...,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1660002453998190593,https://piped.video/watch?v=OkWCf0xJW0c,NaN,NaN,NaN,https://twitter.com/WisnerBaum/status/16600024...,False,35be2b85,en
3,6,Sun Aug 20 13:27:00 +00:00 2023,#Gardasil #HPV regardez les ÈlÈments du procËs...,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1693253363859644790,NaN,NaN,NaN,NaN,https://twitter.com/AcGriffon/status/169325336...,False,ed75e107,fr
4,7,Sat Aug 12 14:25:00 +00:00 2023,Danger pour les ados ‡ la rentrÈe 2023! Le gou...,1,eee6aec2,NaN,NaN,NaN,NaN,NaN,...,NaN,1690368947223171072,https://pgibertie.com/2023/07/13/danger-pour-l...,NaN,NaN,NaN,https://twitter.com/Peck84200/status/169036894...,False,43a73a49,fr


In [2]:
data_used = data['full_text']
data_used.head()

0    Le dimanche c'est #D, comme #Destitution ???? ...
1                 #propagande!! #Vaccination #gardasil
2    Learn about Gardasil HPV Vaccine Multidistrict...
3    #Gardasil #HPV regardez les ÈlÈments du procËs...
4    Danger pour les ados ‡ la rentrÈe 2023! Le gou...
Name: full_text, dtype: object

In [4]:
import re

# Preprocess the text data
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\S*@\S*\s?', '', text)  # Remove emails
    text = re.sub('\'', '', text)  # Remove apostrophes
    text = re.sub('[^a-zA-Z]', ' ', text)  # Remove non-alphabet characters
    text = text.lower()  # Convert to lowercase
    return text

data['preprocessed_text'] = data_used.apply(preprocess_text)

In [5]:
data['preprocessed_text'].head()

0    le dimanche cest  d  comme  destitution       ...
1                  propagande    vaccination  gardasil
2    learn about gardasil hpv vaccine multidistrict...
3     gardasil  hpv regardez les  l ments du proc s...
4    danger pour les ados   la rentr e       le gou...
Name: preprocessed_text, dtype: object

In [6]:
data_english = data[data['language'] == 'en']

In [8]:
data_italian = data[data['language'] == 'it']
data_french = data[data['language'] == 'fr']

In [9]:
import gensim
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Tokenize and remove stopwords
def tokenize(text):
    tokens = gensim.utils.simple_preprocess(text, deacc=True)
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

data_english['tokens'] = data_english['preprocessed_text'].apply(tokenize)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/phuonglinh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/v_/tpg1b9n963d1g2mphq6zhp000000gn/T/ipykernel_86081/2117411335.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_english['tokens'] = data_english['preprocessed_text'].apply(tokenize)


In [16]:
import spacy

# Load spaCy model
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

data_english['lemmas'] = data_english['tokens'].apply(lemmatize)

/var/folders/v_/tpg1b9n963d1g2mphq6zhp000000gn/T/ipykernel_86081/2473408136.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_english['lemmas'] = data_english['tokens'].apply(lemmatize)


In [18]:
import gensim.corpora as corpora

# Create dictionary and corpus
id2word = corpora.Dictionary(data_english['lemmas'])
texts = data_english['lemmas']
corpus = [id2word.doc2bow(text) for text in texts]

In [25]:
lda_model_english = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

In [26]:
topics = lda_model_english.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.049*"s" + 0.036*"new" + 0.021*"that" + 0.018*"order" + 0.015*"woman" + 0.014*"tweet" + 0.014*"bad" + 0.012*"join" + 0.012*"las" + 0.009*"entire"')
(1, '0.047*"video" + 0.027*"pipe" + 0.027*"take" + 0.025*"year" + 0.023*"watch" + 0.023*"see" + 0.017*"etc" + 0.015*"destroy" + 0.015*"never" + 0.015*"every"')
(2, '0.137*"wef" + 0.043*"world" + 0.026*"say" + 0.022*"one" + 0.019*"make" + 0.013*"start" + 0.011*"china" + 0.011*"russia" + 0.010*"another" + 0.009*"not"')
(3, '0.031*"full" + 0.023*"create" + 0.021*"support" + 0.015*"divide" + 0.012*"blackrock" + 0.012*"solution" + 0.011*"sugar" + 0.010*"comply" + 0.010*"vanguard" + 0.009*"yup"')
(4, '0.055*"nwo" + 0.032*"com" + 0.027*"want" + 0.021*"diedsuddenly" + 0.018*"minutecities" + 0.015*"vaccine" + 0.015*"push" + 0.015*"gardasil" + 0.014*"global" + 0.012*"nothing"')
(5, '0.026*"population" + 0.025*"let" + 0.018*"esg" + 0.018*"could" + 0.018*"australia" + 0.015*"puppet" + 0.011*"listen" + 0.011*"buy" + 0.010*"folk" + 0.010*"class"')


In [27]:
from gensim.models import CoherenceModel

# Compute coherence score
coherence_model_lda = CoherenceModel(model=lda_model_english, texts=data_english['lemmas'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.47080604581842794


I have made some changes in the number of topics to identify an optimal number of topics based on the coherence score of each attempt. When the dataset is categorized into 10 topics, the coherence score increased (compared to 3 and 5). Furthermore, 10 is enough for the number of categories. Based on this standard, the number of topics in topic modelling for the next two languages will also be 10. 

In [28]:
data_italian['tokens'] = data_italian['preprocessed_text'].apply(tokenize)

# Load spaCy model
nlp = spacy.load('it_core_news_sm', disable=['parser', 'ner'])

def lemmatize(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

data_italian['lemmas'] = data_italian['tokens'].apply(lemmatize)
id2word = corpora.Dictionary(data_italian['lemmas'])
texts = data_italian['lemmas']
corpus = [id2word.doc2bow(text) for text in texts]
lda_model_italian = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
topics = lda_model_italian.print_topics(num_words=10)
for topic in topics:
    print(topic)

/var/folders/v_/tpg1b9n963d1g2mphq6zhp000000gn/T/ipykernel_86081/4158195037.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_italian['tokens'] = data_italian['preprocessed_text'].apply(tokenize)


(0, '0.010*"lecciones" + 0.010*"ecologismo" + 0.010*"rituales" + 0.010*"farso" + 0.010*"figlio" + 0.010*"atrazina" + 0.010*"satanic" + 0.010*"industria" + 0.010*"namere" + 0.010*"banca"')
(1, '0.112*"agenda" + 0.032*"video" + 0.030*"piped" + 0.023*"il" + 0.016*"potere" + 0.016*"politico" + 0.016*"cos" + 0.015*"loro" + 0.012*"greatreset" + 0.012*"via"')
(2, '0.085*"il" + 0.058*"agenda" + 0.037*"uno" + 0.035*"di" + 0.030*"non" + 0.028*"a il" + 0.026*"di il" + 0.020*"che" + 0.018*"essere" + 0.017*"avere"')
(3, '0.043*"di" + 0.024*"stare" + 0.024*"Usa" + 0.022*"davos" + 0.014*"com" + 0.014*"vaccino" + 0.014*"tutto" + 0.014*"usare" + 0.014*"di il" + 0.010*"schwab"')
(4, '0.077*"si" + 0.036*"agenda" + 0.028*"leggere" + 0.028*"scrivere" + 0.019*"con" + 0.019*"soros" + 0.006*"accoglienza" + 0.006*"ghetto" + 0.006*"liberazione" + 0.006*"franceriots"')
(5, '0.060*"agenda" + 0.027*"il" + 0.024*"globalista" + 0.024*"com" + 0.024*"de" + 0.022*"france" + 0.017*"macrondemission" + 0.017*"macron" + 0.

/var/folders/v_/tpg1b9n963d1g2mphq6zhp000000gn/T/ipykernel_86081/4158195037.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_italian['lemmas'] = data_italian['tokens'].apply(lemmatize)


In [29]:
coherence_model_lda = CoherenceModel(model=lda_model_italian, texts=data_italian['lemmas'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.45179699013097335


In [30]:
data_french['tokens'] = data_french['preprocessed_text'].apply(tokenize)

# Load spaCy model
nlp = spacy.load('fr_core_news_sm', disable=['parser', 'ner'])

def lemmatize(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

data_french['lemmas'] = data_french['tokens'].apply(lemmatize)
id2word = corpora.Dictionary(data_french['lemmas'])
texts = data_french['lemmas']
corpus = [id2word.doc2bow(text) for text in texts]
lda_model_french = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
topics = lda_model_french.print_topics(num_words=10)
for topic in topics:
    print(topic)

/var/folders/v_/tpg1b9n963d1g2mphq6zhp000000gn/T/ipykernel_86081/3955827841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_french['tokens'] = data_french['preprocessed_text'].apply(tokenize)
/var/folders/v_/tpg1b9n963d1g2mphq6zhp000000gn/T/ipykernel_86081/3955827841.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_french['lemmas'] = data_french['tokens'].apply(lemmatize)


(0, '0.030*"sant" + 0.021*"celui" + 0.017*"comment" + 0.017*"votre" + 0.014*"gardasil" + 0.012*"minute" + 0.012*"org" + 0.012*"ministre" + 0.011*"cancer" + 0.011*"mentir"')
(1, '0.071*"le" + 0.044*"de" + 0.031*"pas" + 0.026*"être" + 0.021*"il" + 0.020*"que" + 0.020*"avoir" + 0.019*"un" + 0.019*"ce" + 0.017*"gardasil"')
(2, '0.088*"le" + 0.034*"nouveau" + 0.031*"inefficace" + 0.025*"vaccin" + 0.024*"dangereux" + 0.023*"gardasil" + 0.022*"contre" + 0.021*"com" + 0.021*"ado" + 0.021*"danger"')
(3, '0.030*"odd" + 0.026*"le" + 0.022*"temps" + 0.020*"agender" + 0.015*"autre" + 0.012*"tre" + 0.011*"agenda" + 0.010*"raoult" + 0.009*"tr" + 0.009*"être"')
(4, '0.014*"sident" + 0.013*"promotion" + 0.013*"plainte" + 0.012*"pharmaceutique" + 0.011*"donn" + 0.008*"ind" + 0.008*"er" + 0.008*"merck" + 0.008*"lhumanit" + 0.008*"base"')
(5, '0.098*"de" + 0.097*"le" + 0.043*"gardasil" + 0.030*"un" + 0.026*"et" + 0.023*"pour" + 0.022*"être" + 0.020*"en" + 0.013*"vaccin" + 0.012*"il"')
(6, '0.026*"falloir"

In [31]:
coherence_model_lda = CoherenceModel(model=lda_model_french, texts=data_french['lemmas'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4625807249433322
